In [ ]:
import numpy as np
import pandas as pd

In [ ]:
eps = np.finfo(float).eps

In [ ]:
tennis = pd.read_csv('Sample.csv')

In [ ]:
tennis

In [ ]:
def find_entropy(df):
    target = df.keys()[-1]  # The last dataframe column is the target attribute (playGolf)
    entropy = 0
    values = df[target].unique()

    # for each value in the target playGolf attribute values
    for value in values:
        # ratio of values occurring and entropy
        fraction = df[target].value_counts()[value] / len(df[target])
        entropy += -fraction * np.log2(fraction)
    return entropy

In [ ]:
def find_entropy_attribute(df, attribute):
    target = df.keys()[-1]
    target_variables = df[target].unique()  # unique values in target playGolf attribute (Yes, No)
    variables = df[attribute].unique() # Identify Sunny, Overcast, Rainy

    # attribute entropy          # Variables=[sunny, sunny....5, overcast1.....overcast4, Rainy1...Ra5 ]
    entropy2 = 0
    # for each attribute value in attribute values
    for variable in variables:
        # value entropy
        entropy = 0
        # for each target value in target values (yes/no)
        for target_variable in target_variables:
            # frequency of attribute and target values (boolean indexing, pandas dataframe filtering)
            num = len(df[attribute][df[attribute] == variable][df[target] == target_variable])
            den = len(df[attribute][df[attribute] == variable])
            fraction = num / (den + eps)
            entropy += -fraction * np.log2(fraction + eps)
        fraction2 = den / len(df)
        entropy2 += -fraction2 * entropy
    return abs(entropy2)

In [ ]:
def bestClassifier(df):
    # Entropy_att = []
    # information gain array for all attributes
    IG = []
    # for all attributes excluding target
    for key in df.keys()[:-1]:
        # Entropy_att.append(find_entropy_attribute(df,key))
        # calculate and record information gain value
        IG.append(find_entropy(df) - find_entropy_attribute(df, key)) #0.940 -0.693= 0.247
    return df.keys()[:-1][np.argmax(IG)]  # IG[0.247, 0.029, 0.152, 0.048 ]

In [ ]:
def get_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop=True)

In [ ]:
def ID3split(df, tree=None):
    target = df.keys()[-1]

    # Here we build our decision tree

    # Get attribute with maximum information gain
    node = bestClassifier(df) # 0.247

    # Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attributeValues = np.unique(df[node])

    # Create an empty dictionary to create tree (recursive-friendly definition)
    if tree is None:               #  Outlook ->root node attribute
        tree = {}
        tree[node] = {}

    # following loop recursively calls ID3split to create and add to the tree
    # it runs till the tree is pure (leaf (result) node branches are added to the tree)
    for value in attributeValues:
        # get the subtable from current node based on the value
        subtable = get_subtable(df, node, value)
        # get the most common target value in the subtable
        targetValues, counts = np.unique(subtable[target], return_counts=True)

        # if the subtable is empty, assign the leaf node to the most common target value
        if len(counts) == 1:
            tree[node][value] = targetValues[0]
        else:
            # recursively call ID3 to create subtrees
            tree[node][value] = ID3split(subtable)  # Calling the function recursively

    return tree

In [ ]:
decisionTree = ID3split(dataframe)
print(decisionTree)